### Install dependencies

In [ ]:
%%bash
# git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

### Download dataset (OxfordPets)

In [ ]:
%%bash
cd ../../datasets/data
mkdir oxford_pets
cd oxford_pets
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
gdown 1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs
tar -zxvf images.tar.gz
tar -zxvf annotations.tar.gz

### Run MaPLe

In [ ]:
%%bash
# seed=1
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 1 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 1 ../../datasets/data/

In [ ]:
%%bash
# seed=2
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 2 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 2 ../../datasets/data/

In [ ]:
%%bash
# seed=3
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 3 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 3 ../../datasets/data/

## Prompt Learning

In [1]:
%%bash
bash scripts/maple_prompt_scene/base2new_train_maple_prompt_scene.sh replica 1 ../../datasets/data/

Run this job and save the output to output/base2new/train_base/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1
Error while terminating subprocess (pid=2879446): 


In [3]:
%%bash
bash scripts/maple_prompt_scene/base2new_test_maple_prompt_scene.sh replica 1 ../../datasets/data/

Evaluating model
Runing the first phase job and save the output to output/base2new/test_new/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Error while terminating subprocess (pid=2879966): 


In [16]:
%%bash
bash scripts/maple_prompt_scene/xd_train_maple_prompt_scene.sh replica 1 ../../experiments/merged_pipline/run_2025-01-27-03-27-30/

Run this job and save the output to output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_10000shots/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: False
head: 
load_epoch: None
model_dir: 
no_train: False
opts: ['DATASET.NUM_SHOTS', '10000']
output_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_10000shots/seed1
resume: 
root: ../../experiments/merged_pipline/run_2025-01-27-03-27-30/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIF

test: [<datasets.replica.Datum_feature object at 0x7f63f1089e70>, <datasets.replica.Datum_feature object at 0x7f63f1089a50>, <datasets.replica.Datum_feature object at 0x7f63f1089a80>, <datasets.replica.Datum_feature object at 0x7f63f1089de0>, <datasets.replica.Datum_feature object at 0x7f63f108a4d0>]
-----------------------------------------------------------
Label range: 0-17
Unique labels: Counter({2: 2, 3: 2, 0: 1, 1: 1, 17: 1, 10: 1, 11: 1, 4: 1})
Label range: 5-14
Unique labels: Counter({12: 1, 13: 1, 14: 1, 5: 1, 6: 1, 7: 1})
Label range: 8-16
Unique labels: Counter({15: 1, 16: 1, 14: 1, 8: 1, 9: 1})
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.457

100%|██████████| 1/1 [00:00<00:00,  2.87it/s]


output: tensor([[ 5.4258,  7.3125,  6.4453,  7.1836,  7.6797,  6.6914,  6.1211,  5.8203,
          5.9492,  6.4180,  6.5312,  6.9180,  5.6641,  5.7031,  6.2188,  7.6602,
          6.3398,  7.5117],
        [ 6.7695,  8.9844,  7.6484,  8.5000,  9.1406,  7.4414,  7.2422,  6.6094,
          7.3906,  7.6172,  8.0312,  8.2422,  7.3789,  7.1016,  8.1875,  9.0938,
          8.1250,  9.1484],
        [ 7.7422,  9.8047,  9.3984,  8.6797, 10.8047,  8.8125,  8.2578,  8.8906,
          8.6016,  8.3906,  6.8828,  9.8906,  8.4375,  9.4062,  8.7656,  9.1719,
          7.5352,  8.1328],
        [ 6.7695,  8.8828,  7.6289,  8.4531,  9.0156,  7.5156,  7.1797,  6.6875,
          7.3828,  7.5117,  7.9297,  8.2734,  7.3672,  7.0742,  8.2188,  9.1797,
          8.0391,  9.1406],
        [ 8.1406, 10.1016,  9.6094,  8.9297, 11.0156,  8.9375,  8.8516,  9.1719,
          9.2734,  8.2734,  6.7773, 10.4453,  9.4141, 10.2266,  9.1875,  9.0938,
          7.6641,  7.9336]], device='cuda:0', dtype=torch.float16)
lab

In [19]:
%%bash
bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../experiments/merged_pipline/run_2025-01-27-03-27-30/

Run this job and save the output to output/evaluation/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_10000shots/replica/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_10000shots/seed1
no_train: False
opts: []
output_dir: output/evaluation/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_10000shots/replica/seed1
resume: 
root: ../../experiments/merged_pipline/run_2025-01-27-03-27-30/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 1

test: [<datasets.replica.Datum_feature object at 0x7f44122d1540>, <datasets.replica.Datum_feature object at 0x7f44122d21a0>, <datasets.replica.Datum_feature object at 0x7f44122d1d20>, <datasets.replica.Datum_feature object at 0x7f44122d17e0>, <datasets.replica.Datum_feature object at 0x7f44122d1960>]
-----------------------------------------------------------
Label range: 0-17
Unique labels: Counter({2: 2, 3: 2, 0: 1, 1: 1, 17: 1, 10: 1, 11: 1, 4: 1})
Label range: 5-14
Unique labels: Counter({12: 1, 13: 1, 14: 1, 5: 1, 6: 1, 7: 1})
Label range: 8-16
Unique labels: Counter({15: 1, 16: 1, 14: 1, 8: 1, 9: 1})
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.457

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


output: tensor([[ 5.4258,  7.3125,  6.4453,  7.1836,  7.6797,  6.6914,  6.1211,  5.8203,
          5.9492,  6.4180,  6.5312,  6.9180,  5.6641,  5.7031,  6.2188,  7.6602,
          6.3398,  7.5117],
        [ 6.7695,  8.9844,  7.6484,  8.5000,  9.1406,  7.4414,  7.2422,  6.6094,
          7.3906,  7.6172,  8.0312,  8.2422,  7.3789,  7.1016,  8.1875,  9.0938,
          8.1250,  9.1484],
        [ 7.7422,  9.8047,  9.3984,  8.6797, 10.8047,  8.8125,  8.2578,  8.8906,
          8.6016,  8.3906,  6.8828,  9.8906,  8.4375,  9.4062,  8.7656,  9.1719,
          7.5352,  8.1328],
        [ 6.7695,  8.8828,  7.6289,  8.4531,  9.0156,  7.5156,  7.1797,  6.6875,
          7.3828,  7.5117,  7.9297,  8.2734,  7.3672,  7.0742,  8.2188,  9.1797,
          8.0391,  9.1406],
        [ 8.1406, 10.1016,  9.6094,  8.9297, 11.0156,  8.9375,  8.8516,  9.1719,
          9.2734,  8.2734,  6.7773, 10.4453,  9.4141, 10.2266,  9.1875,  9.0938,
          7.6641,  7.9336]], device='cuda:0', dtype=torch.float16)
lab

In [ ]:

trainer = build_trainer(cfg)
trainer.load_model(args.model_dir, epoch=args.load_epoch)